In [ ]:
from keras.datasets import mnist
import numpy as np
import math
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = mnist.load_data() # data-set load

row_num, col_num = 28, 28
#x_train = x_train.reshape(x_train.shape[0], row_num, col_num, 1)
#x_test = x_test.reshape(x_test.shape[0], row_num, col_num, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# x_train plot

plot_index = 6000
plt.imshow(x_train[plot_index], cmap='gray')
plt.title('%i' % y_train[plot_index])  
plt.show()

In [ ]:
def fixed_point_hex_generate(data, int_bit, point_bit, merge):
  hex_map = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']

  pre_point = data - data % 1
  post_point = data % 1

  
  result = [['' for col in range(28)] for row in range(28)]

  while point_bit >= 4:
    point_bit -= 4
    post_point = post_point * 16
    for i in range(row_num):
      for j in range(col_num):
        result[i][j] = result[i][j] + hex_map[int(post_point[i][j])]
    post_point = post_point % 1
  if point_bit != 0:
    post_point = post_point * (2 ** point_bit)
    for i in range(row_num):
      for j in range(col_num):
        result[i][j] = result[i][j] + hex_map[int(post_point[i][j]) * (2 ** (4 - point_bit))]

  '''try'''

  while int_bit >= 4:
    int_bit -= 4
    for i in range(row_num):
      for j in range(col_num):
        result[i][j] = hex_map[int(pre_point[i][j] % 16)] + result[i][j]
    pre_point = (pre_point / 16) % 1
  if int_bit != 0:
    for i in range(row_num):
      for j in range(col_num):
        result[i][j] = hex_map[int(pre_point[i][j] % 16)] + result[i][j]
  return result

In [ ]:
'''

Modify these variable to change how much bit you want reserve

'''

post_point_bit = 13 # bit after point
pre_point_bit = 12 # bit before point

post_point_hex_size = math.ceil(post_point_bit/4) # hex size after point
pre_point_hex_size = math.ceil(pre_point_bit/4) # hex size before point

In [ ]:
result = fixed_point_hex_generate(x_train[plot_index], pre_point_bit,  post_point_bit, 1);

In [ ]:
'''test post_point'''

test_post_point = x_train[plot_index] - x_train[plot_index] % (2 ** -(post_point_bit))
for i in range(row_num):
  for j in range(col_num):
    test_result = 0
    for int_index in range(pre_point_hex_size -1,  -1, -1):
      test_result += int(result[i][j][int_index], 16) * (16 ** (pre_point_hex_size - int_index - 1))
    for float_index in range(post_point_hex_size):
      test_result += int(result[i][j][float_index + pre_point_hex_size], 16) * (1.0 / (16 ** (float_index + 1)))
    if test_post_point[i][j] != test_result:
      print(test_post_point[i][j], test_result)